In [23]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

from paddleocr import PaddleOCR


from PIL import Image
from paddleocr import TextRecognition
from paddleocr import TextDetection

from helpers.inspector import inspect_object


In [28]:
# Bildpfad
image_path = "Schneckenwelle.jpg"
image = Image.open(image_path).convert("RGB")
img_np = np.array(image)


In [31]:
inspect_object(image, name="img_np")


--- Inspecting: img_np ---
Type: <class 'PIL.Image.Image'>
Attributes:
 - alpha_composite: <class 'method'>
--- Inspecting: img_np.alpha_composite ---
Type: <class 'method'>
Attributes:


 - apply_transparency: <class 'method'>
--- Inspecting: img_np.apply_transparency ---
Type: <class 'method'>
Attributes:


 - close: <class 'method'>
--- Inspecting: img_np.close ---
Type: <class 'method'>
Attributes:


 - convert: <class 'method'>
--- Inspecting: img_np.convert ---
Type: <class 'method'>
Attributes:


 - copy: <class 'method'>
--- Inspecting: img_np.copy ---
Type: <class 'method'>
Attributes:


 - crop: <class 'method'>
--- Inspecting: img_np.crop ---
Type: <class 'method'>
Attributes:


 - draft: <class 'method'>
--- Inspecting: img_np.draft ---
Type: <class 'method'>
Attributes:


 - effect_spread: <class 'method'>
--- Inspecting: img_np.effect_spread ---
Type: <class 'method'>
Attributes:


 - entropy: <class 'method'>
--- Inspecting: img_np.entropy ---
Type: <class 'method'>
Attr

In [15]:
# Text Detection Modell (PP-OCRv5)
detector = TextDetection(model_name="PP-OCRv5_server_det")

# Run detection
detection_result = detector.predict(input=[img_np])
print(f"Gefundene Textboxen: {detection_result}")

for res in detection_result:
    res.print()
    res.save_to_img(save_path="./output/det/")
    res.save_to_json(save_path="./output/det/res.json")
    
    
det_res = detection_result[0]


MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in /Users/gerhardkarbeutz/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': None, 'page_index': None, 'dt_polys': array([[[879, 897],
        ...,
        [879, 907]],

       ...,

       [[944,  85],
        ...,
        [944, 112]]], dtype=int16), 'dt_scores': [0.7945580862772961, 0.9304698371949295, 0.7561921024698985, 0.7281057972460986, 0.6177545873532918, 0.9268560577239398, 0.91293188696198, 0.9883202606317948, 0.9056529998779297, 0.8352959785682846, 0.7406655207315149, 0.9759778354905153, 0.8639507440863119, 0.9579723698319867, 0.9449299596482889, 0.8859441453116198, 0.6722485486079346, 0.8346600553227795, 0.9766052586043844, 0.9586822539129282, 0.9713346474512946, 0.8740434740971336, 0.9100476460459096, 0.9166111927479506, 0.8790461783781648, 0.8158156524101893, 0.7580048458029827, 0.9308460106558915, 0.9785615130359135, 0.9716165825258941, 0.9519295502115379, 0.9490360067015694, 0.9714873020350934, 0.8731619566506958, 0.9550017527750005, 0.9166160068091224, 0.7894653652540662, 0.7852050672805534, 0.9324886446353048, 0.93795921

Gefundene Textboxen: [{'input_path': None, 'page_index': None, 'input_img': array([[[255, ..., 255],
        ...,
        [255, ..., 255]],

       ...,

       [[255, ..., 255],
        ...,
        [255, ..., 255]]], dtype=uint8), 'dt_polys': array([[[879, 897],
        ...,
        [879, 907]],

       ...,

       [[944,  85],
        ...,
        [944, 112]]], dtype=int16), 'dt_scores': [0.7945580862772961, 0.9304698371949295, 0.7561921024698985, 0.7281057972460986, 0.6177545873532918, 0.9268560577239398, 0.91293188696198, 0.9883202606317948, 0.9056529998779297, 0.8352959785682846, 0.7406655207315149, 0.9759778354905153, 0.8639507440863119, 0.9579723698319867, 0.9449299596482889, 0.8859441453116198, 0.6722485486079346, 0.8346600553227795, 0.9766052586043844, 0.9586822539129282, 0.9713346474512946, 0.8740434740971336, 0.9100476460459096, 0.9166111927479506, 0.8790461783781648, 0.8158156524101893, 0.7580048458029827, 0.9308460106558915, 0.9785615130359135, 0.9716165825258941, 0.9519

In [17]:
print("Verfügbare Attribute:")
for attr in dir(det_res):
    if not attr.startswith("_"):
        print(f" - {attr}")

Verfügbare Attribute:
 - clear
 - copy
 - fromkeys
 - get
 - img
 - items
 - json
 - keys
 - pop
 - popitem
 - print
 - save_all
 - save_to_img
 - save_to_json
 - setdefault
 - str
 - update
 - values


In [18]:
print(det_res.keys())


dict_keys(['input_path', 'page_index', 'input_img', 'dt_polys', 'dt_scores'])


In [19]:
image = det_res["input_img"]         # ← Bild als NumPy-Array
boxes = det_res["dt_polys"]          # ← Liste der Textboxen
scores = det_res["dt_scores"]        # ← Liste der Confidence-Werte


In [20]:


def crop_boxes(image, boxes):
    crops = []
    for box in boxes:
        # Hole die Koordinaten (int)
        box = np.array(box).astype(np.int32)

        # Berechne Bounding-Box als Rechteck
        rect = cv2.boundingRect(box)
        x, y, w, h = rect

        # Crop aus Bild schneiden
        crop = image[y:y+h, x:x+w]
        crops.append(crop)
    return crops


In [21]:
crops = crop_boxes(image, boxes)
print(f"{len(crops)} Textregionen extrahiert.")


115 Textregionen extrahiert.


In [22]:
recognizer = TextRecognition(model_name="PP-OCRv5_server_rec")
output = recognizer.predict(input=crops, batch_size=1)

# Jeder Crop separat speichern
for idx, res in enumerate(output):
    res.print()

    save_folder = f"./output/rec/crop_{idx}"
    os.makedirs(save_folder, exist_ok=True)

    res.save_to_img(save_path=save_folder)
    res.save_to_json(save_path=os.path.join(save_folder, "res.json"))


MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-OCRv5_server_rec), the model files will be automatically downloaded and saved in /Users/gerhardkarbeutz/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': None, 'page_index': None, 'rec_text': '', 'rec_score': 0.0}}
There is not input file name as reference for name of saved result file. So the saved result file would be named with timestamp and random number: `1751465641_8808`.
{'res': {'input_path': None, 'page_index': None, 'rec_text': 'Da', 'rec_score': 0.4286233186721802}}
There is not input file name as reference for name of saved result file. So the saved result file would be named with timestamp and random number: `1751465641_8557`.
{'res': {'input_path': None, 'page_index': None, 'rec_text': 'Anderung', 'rec_score': 0.7714438438415527}}
There is not input file name as reference for name of saved result file. So the saved result file would be named with timestamp and random number: `1751465641_2701`.
{'res': {'input_path': None, 'page_index': None, 'rec_text': '', 'rec_score': 0.0}}
There is not input file name as reference for name of saved result file. So the saved result file would be named with timestam

ValueError: font size must be greater than 0, not 0

In [ ]:
print("Verfügbare Attribute:")
for attr in dir(output):
    if not attr.startswith("_"):
        print(f" - {attr}")

In [ ]:
print(output)

In [ ]:
results_list = []
for idx, res in enumerate(output):
    results_list.append({
        "index": idx,
        "text": res['rec_text'],
        "score": res['rec_score']
    })

import pandas as pd
pd.DataFrame(results_list).to_csv("./output/rec_summary.csv", index=False)
